In [ ]:
# Get the list of the names

import glob

The_list = []
sls = glob.glob("Path/To/Thumbnails/*.png")
for i in sls:
    The_list.append(str(i[11:-4]))


len(The_list)

In [ ]:
import math
import os
import fast
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Select GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## if two levels of patches are to be used, if only one is used lvl3_B_p_size can be ignored
lvl3_B_p_size = 256 * 3
lvl3_s_p_size = 256
Th_B_p_size = 51


# get the sizes:

Slides_test = The_list[0:20]

SlNames_train = The_list[20:76]

SlNames_valid = The_list[76:97]


def get_sizes_into_set(SlideNames, SlideDir, ThumbDir, EmptySet):
    for filename in SlideNames:
        if not isinstance(filename, str):
            filename = filename.numpy().decode("utf-8")

        importer = fast.WholeSlideImageImporter.create(SlideDir + filename + ".vsi")
        wsi = importer.runAndGetOutputData()
        plane_height5 = wsi.getLevelHeight(5)
        plane_width5 = wsi.getLevelWidth(5)
        plane_height3 = wsi.getLevelHeight(3)
        plane_width3 = wsi.getLevelWidth(3)
        thumbnail = cv2.imread(ThumbDir + filename + ".png")
        thumbnail_height = thumbnail.shape[0]
        thumbnail_width = thumbnail.shape[1]
        # print(filename+': ', [plane_height5, plane_width5, plane_height3, plane_width3, thumbnail_height, thumbnail_width])
        EmptySet[filename] = [
            plane_height5,
            plane_width5,
            plane_height3,
            plane_width3,
            thumbnail_height,
            thumbnail_width,
        ]


def get_initial_starting_pos(Slides):

    # finding the last posion on the thumbnail to stop forming patches
    last_pos_X = []
    last_pos_Y = []
    for Nmbr, filename in enumerate(Slides):
        last_pos_X.append(lvl3_B_p_size * (math.floor(Slides[filename][3] / lvl3_B_p_size)) - 256)
        last_pos_Y.append(lvl3_B_p_size * (math.floor(Slides[filename][2] / lvl3_B_p_size)) - 256)

    # finding the starting positions of X for lvl3_B_patches
    Start_pos_X_small = []
    Start_pos_Y_small = []

    Start_pos_X_big = []
    Start_pos_Y_big = []

    for indx, filename in enumerate(Slides):
        Start_pos_X_small.append(list(range((256) * 2, last_pos_X[indx], int(lvl3_B_p_size / 3))))
        Start_pos_Y_small.append(list(range((256) * 2, last_pos_Y[indx], int(lvl3_B_p_size / 3))))

    for i, _ in enumerate(Start_pos_X_small):
        Start_pos_X_big.append([])
        for j in Start_pos_X_small[i]:
            Start_pos_X_big[i].append(round(j - 256 * 1.5))

    for i, _ in enumerate(Start_pos_Y_small):
        Start_pos_Y_big.append([])
        for j in Start_pos_Y_small[i]:
            Start_pos_Y_big[i].append(round(j - 256 * 1.5))

    # getting the fifth level starting positions:

    Start_pos_X_fifthLevel = []
    Start_pos_Y_fifthLevel = []

    for indx, filename in enumerate(Slides):
        Start_pos_X_fifthLevel.append([])
        for ind, _ in enumerate(Start_pos_X_big[indx]):
            Start_pos_X_fifthLevel[indx].append(
                round((Start_pos_X_big[indx][ind] / Slides[filename][3]) * Slides[filename][1])
            )
        Start_pos_Y_fifthLevel.append([])
        for ind2, _ in enumerate(Start_pos_Y_big[indx]):
            Start_pos_Y_fifthLevel[indx].append(
                round((Start_pos_Y_big[indx][ind2] / Slides[filename][2]) * Slides[filename][0])
            )

    # # get the corresponding Th_B_patch starting positions from Start_pos_X and Start_pos_Y, so you can check the tissue and GT

    Th_Start_pos_X_small = []
    Th_Start_pos_Y_small = []

    Th_Start_pos_X_big = []
    Th_Start_pos_Y_big = []

    for indx, filename in enumerate(Slides):
        Th_Start_pos_X_small.append([])
        for ind, _ in enumerate(Start_pos_X_small[indx]):
            Th_Start_pos_X_small[indx].append(
                round((Start_pos_X_small[indx][ind] / Slides[filename][3]) * Slides[filename][5])
            )  # we add thr 20 to get the central small patch not the big one then the patch size should be 10px
        Th_Start_pos_Y_small.append([])
        for ind2, _ in enumerate(Start_pos_Y_small[indx]):
            Th_Start_pos_Y_small[indx].append(
                round((Start_pos_Y_small[indx][ind2] / Slides[filename][2]) * Slides[filename][4])
            )  # we add thr 20 to get the central small patch not the big one then the patch size should be 10px

        Th_Start_pos_X_big.append([])
        for ind, _ in enumerate(Start_pos_X_fifthLevel[indx]):
            Th_Start_pos_X_big[indx].append(
                round((Start_pos_X_fifthLevel[indx][ind] / Slides[filename][1]) * Slides[filename][5])
            )  # we add thr 20 to get the central small patch not the big one then the patch size should be 10px
        Th_Start_pos_Y_big.append([])
        for ind2, _ in enumerate(Start_pos_Y_fifthLevel[indx]):
            Th_Start_pos_Y_big[indx].append(
                round((Start_pos_Y_fifthLevel[indx][ind2] / Slides[filename][0]) * Slides[filename][4])
            )  # we add thr 20 to get the central small patch not the big one then the patch size should be 10px

    return (
        Start_pos_X_small,
        Start_pos_Y_small,
        Start_pos_X_big,
        Start_pos_Y_big,
        Start_pos_X_fifthLevel,
        Start_pos_Y_fifthLevel,
        Th_Start_pos_X_small,
        Th_Start_pos_Y_small,
        Th_Start_pos_X_big,
        Th_Start_pos_Y_big,
    )


Slides_train = {}

get_sizes_into_set(SlNames_train, "Path/To/WSIs/", "/Path/To/Thumbnails/", Slides_train)


Slides_valid = {}

get_sizes_into_set(SlNames_valid, "Path/To/WSIs/", "/Path/To/Thumbnails/", Slides_valid)


(
    Start_pos_X_small_train,
    Start_pos_Y_small_train,
    Start_pos_X_big_train,
    Start_pos_Y_big_train,
    Start_pos_X_fifthLevel_train,
    Start_pos_Y_fifthLevel_train,
    Th_Start_pos_X_small_train,
    Th_Start_pos_Y_small_train,
    Th_Start_pos_X_big_train,
    Th_Start_pos_Y_big_train,
) = get_initial_starting_pos(Slides_train)
(
    Start_pos_X_small_valid,
    Start_pos_Y_small_valid,
    Start_pos_X_big_valid,
    Start_pos_Y_big_valid,
    Start_pos_X_fifthLevel_valid,
    Start_pos_Y_fifthLevel_valid,
    Th_Start_pos_X_small_valid,
    Th_Start_pos_Y_small_valid,
    Th_Start_pos_X_big_valid,
    Th_Start_pos_Y_big_valid,
) = get_initial_starting_pos(Slides_valid)

In [ ]:
# Visualizing the patch results

image = cv2.imread("/Path/To/Thumbnails/" + SlNames_train[0] + ".png")

for start_y in Th_Start_pos_Y_small_train[0]:
    for start_x in Th_Start_pos_X_small_train[0]:
        end_x = start_x + 10
        end_y = start_y + 10
        cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 1)  # Draws a green rectangle

for start_y in Th_Start_pos_Y_big_train[0][0:1]:
    for start_x in Th_Start_pos_X_big_train[0]:
        end_x = start_x + 40
        end_y = start_y + 40
        cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (255, 0, 0), 1)  # Draws a green rectangle

plt.imshow(image)
plt.show()

In [ ]:
# create tissue masks

# Path to the folder containing the image thumbnails
folder_path = "/Path/To/Thumbnails/"

# Path to the folder where you want to save the masks
save_path = "/Path/To/TissueMask/Results/"

# Create the save folder if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Iterate over the images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)

        # Apply Gaussian blur to the image
        blurred = cv2.GaussianBlur(image, (15, 15), 0)

        # Convert the blurred image to grayscale
        gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

        # Threshold the image to obtain the mask
        _, mask = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY_INV)

        # Save the mask
        mask_filename = os.path.splitext(filename)[0] + ".png"
        mask_path = os.path.join(save_path, mask_filename)
        cv2.imwrite(mask_path, mask)

print("Masks saved successfully!")

In [ ]:
image = cv2.imread("/Path/To/Thumbnails/" + SlNames_train[0] + ".png")
tissue = cv2.imread("/Path/To/TissueMask/Results/" + SlNames_train[0] + ".png")

for indx1, start_y in enumerate(Th_Start_pos_Y_small_train[0]):
    for indx2, start_x in enumerate(Th_Start_pos_X_small_train[0]):
        end_x = start_x + 10
        end_y = start_y + 10

        tissue_patch = tissue[start_y:end_y, start_x:end_x, :]
        if len(tissue_patch[tissue_patch > 40]) > 0.3 * len(tissue_patch[tissue_patch < 300]):
            cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 0, 0), 4)  # Draws a green rectangle
            # patches = load_patches([Start_pos_X[0][indx2]+512], [Start_pos_Y[0][indx1]+512], "D:/WSIs/WSIs/" + 'HUS_L_005_IC' + ".vsi", 3, 256)
            # img = np.array(patches[0])
            # cv2.imwrite('./test/'+'HUS_L_005_IC'+'X'+str(indx2)+'Y'+str(indx1)+'.png', img)


plt.imshow(image)
plt.show()

In [ ]:
## getting the final positions


def get_clusters_label(arr):

    unique_values, counts = np.unique(arr, return_counts=True)

    # Find the index of the most dominant value
    dominant_index = np.argmax(counts)

    # Check if the most dominant value is zero
    if unique_values[dominant_index] == 0:
        # Find the next most dominant value by excluding zero from consideration
        non_zero_counts = counts[unique_values != 0]
        non_zero_values = unique_values[unique_values != 0]
        next_dominant_index = np.argmax(non_zero_counts)
        dominant_value = non_zero_values[next_dominant_index]
    else:
        # If the most dominant value is not zero, return it
        dominant_value = unique_values[dominant_index]

    return dominant_value


def get_final_starting_pos(
    startingPositions,
    gts,
    Slides,
    Th_Start_pos_Y_small,
    Th_Start_pos_X_small,
    Start_pos_X_fifthLevel,
    Start_pos_Y_fifthLevel,
    Start_pos_X_small,
    Start_pos_Y_small,
):

    for indx, filename in enumerate(Slides):

        # startingPositions.append([])
        # gts.append([])
        print(indx)

        name = str(list(Slides.keys())[indx])

        tissue = cv2.imread("/Path/To/TissueMask/Results/" + name + ".png")
        gt = cv2.imread("/Path/To/GTs/" + name + ".png")
        clusters = np.load("/Path/To/ClusterResults/" + name + ".npy")

        for indx1, start_y in enumerate(Th_Start_pos_Y_small[indx]):
            for indx2, start_x in enumerate(Th_Start_pos_X_small[indx]):
                end_x = start_x + 10
                end_y = start_y + 10

                tissue_patch = tissue[start_y:end_y, start_x:end_x, :]
                gt_patch = gt[start_y:end_y, start_x:end_x, :]

                if len(tissue_patch[tissue_patch > 40]) > 0.6 * len(tissue_patch[tissue_patch < 300]):
                    # patches = load_patches([Start_pos_X[indx][indx2]+512], [Start_pos_Y[indx][indx1]+512], "D:/WSIs/WSIs/" + name + ".vsi", 3, 256)
                    # img = np.array(patches[0])
                    # starting positions = [name, lvl5, lvl3]
                    label = get_clusters_label(clusters)
                    startingPositions.append(
                        [
                            "Path/To/WSIs/" + name + ".vsi",
                            [Start_pos_X_fifthLevel[indx][indx2]],
                            [Start_pos_Y_fifthLevel[indx][indx1]],
                            [Start_pos_X_small[indx][indx2]],
                            [Start_pos_Y_small[indx][indx1]],
                            label,
                        ]
                    )

                    if len(gt_patch[gt_patch > 25]) > 0.25 * len(gt_patch[gt_patch < 300]):
                        # cv2.imwrite('./test/tumor/'+'HUS_L_005_IC'+'X'+str(indx2)+'Y'+str(indx1)+'.png', img)
                        gts.append(1)
                        # cv2.imwrite('./test/tumor/'+name+'X'+str(indx2)+'Y'+str(indx1)+'.png', gt_patch)

                    elif len(gt_patch[gt_patch < 20]) > 0.90 * len(gt_patch[gt_patch < 300]):
                        gts.append(0)
                        # cv2.imwrite('./test/healthy/'+name+'X'+str(indx2)+'Y'+str(indx1)+'.png', gt_patch)

                    else:
                        startingPositions.pop()

                    # if len(gt_patch[gt_patch < 10]) > 0.8 * len(gt_patch[gt_patch < 300]):
                    # cv2.imwrite('./test/healthy/'+'HUS_L_005_IC'+'X'+str(indx2)+'Y'+str(indx1)+'.png', img)


# starting positions = [[name, lvl5X, lvl5Y, lvl3X, lvl3Y]]
startingPositions_train = []
gts_train = []


startingPositions_valid = []
gts_valid = []


get_final_starting_pos(
    startingPositions_train,
    gts_train,
    Slides_train,
    Th_Start_pos_Y_small_train,
    Th_Start_pos_X_small_train,
    Start_pos_X_fifthLevel_train,
    Start_pos_Y_fifthLevel_train,
    Start_pos_X_small_train,
    Start_pos_Y_small_train,
)
get_final_starting_pos(
    startingPositions_valid,
    gts_valid,
    Slides_valid,
    Th_Start_pos_Y_small_valid,
    Th_Start_pos_X_small_valid,
    Start_pos_X_fifthLevel_valid,
    Start_pos_Y_fifthLevel_valid,
    Start_pos_X_small_valid,
    Start_pos_Y_small_valid,
)

In [ ]:
# export starting positions

import pickle

# Saving the lists to a file
with open("PositionsAndGts_train.pkl", "wb") as file:
    pickle.dump(startingPositions_train, file)
    pickle.dump(gts_train, file)


with open("PositionsAndGts_valid.pkl", "wb") as file:
    pickle.dump(startingPositions_valid, file)
    pickle.dump(gts_valid, file)